# Simple Workflow Flow Sample

This implements the sample workflow from the SWF tutorial

In [ ]:
import boto3
from botocore.client import Config
import uuid
botoConfig = Config(connect_timeout=50, read_timeout=70)
client = boto3.client('swf', config=botoConfig)

In [ ]:
workflowDomain = 'TutorialWorkflowDomain2'
workflow = 'TutorialWorkflow'
tasklist = 'TutorialTasks'

activities = ['get_contact_activity','subscribe_topic_activity','wait_for_confirmation_activity',
             'send_result_activity']

In [ ]:
############################################
## DEFINITION
############################################

In [ ]:
# To me master of your domain, you must first register it.
response = client.register_domain(
    name=workflowDomain,
    description='Tutorial workflow domain',
    workflowExecutionRetentionPeriodInDays='1'
)

print response

In [ ]:
response = client.register_workflow_type(
    domain=workflowDomain,
    name=workflow,
    version='1',
    description='Tutorial workflow',
    defaultTaskStartToCloseTimeout='NONE',
    defaultExecutionStartToCloseTimeout='900',
    defaultTaskList={
        'name': tasklist
    },
    defaultChildPolicy='TERMINATE'
)

print response

In [ ]:


for activity in activities:
    response = client.register_activity_type(
        domain=workflowDomain,
        name=activity,
        version='1',
        defaultTaskList={
            'name': tasklist
        }
    )
    
    print response

In [ ]:
############################################
## EXECUTION
############################################

In [ ]:
def schedule_activity(task_token, activity_name, activity_id):
    response = client.respond_decision_task_completed(
        taskToken=task_token,
        decisions=[
          {
            'decisionType': 'ScheduleActivityTask',
            'scheduleActivityTaskDecisionAttributes': {
                'activityType':{
                    'name': activity_name,
                    'version': '1'
                    },
                'activityId': activity_id,
                'input': '',
                'scheduleToCloseTimeout': 'NONE',
                'scheduleToStartTimeout': 'NONE',
                'startToCloseTimeout': 'NONE',
                'heartbeatTimeout': 'NONE',
                'taskList': {'name': tasklist},
            }
          }
        ]
      )

    print response

In [ ]:
def poll_and_complete_task():
    
    task = client.poll_for_activity_task(
        domain=workflowDomain,
        taskList={'name': tasklist},
        identity='worker-1'
    )

    if 'taskToken' not in task:
        print "Poll timed out, no new task.  Repoll"
        return
    else:
        print "New task arrived"

    client.respond_activity_task_completed(
        taskToken=task['taskToken'],
        result='success'
    )

    print "Task Done"




In [ ]:
def print_pending_activity_task_count():
    response = client.count_pending_activity_tasks(
        domain=workflowDomain,
        taskList={
            'name': tasklist
        }
    )

    print 'Pending activity task count: {}'.format(response['count'])


In [ ]:
def print_pending_descision_task_count():
    response = client.count_pending_decision_tasks(
        domain=workflowDomain,
        taskList={
            'name': tasklist
        }
    )

    print 'Pending decision task count: {}'.format(response['count'])

In [ ]:
def get_decision_task(decider_id):
    newTask = client.poll_for_decision_task(
        domain=workflowDomain,
        taskList={'name': tasklist},
        identity=decider_id,
        reverseOrder=False)
    
    if 'taskToken' not in newTask:
        print 'No task available'
    
    return newTask

In [ ]:
# Arbitrary choice for decider id
decider_id = 'decider-1'

In [ ]:
# Is there a task available?
newTask = get_decision_task(decider_id)

print newTask

In [ ]:
# Instantiate the worflow
response = client.start_workflow_execution(
  domain=workflowDomain,
  workflowId='wf-1',
  workflowType={
    "name": 'TutorialWorkflow',
    "version": '1'
  },
  taskList={
      'name': tasklist
  },
  input=''
)

print response

In [ ]:
# Is there a task available?
newTask = get_decision_task(decider_id)

print newTask

In [ ]:
events = newTask['events']
print events

In [ ]:
eventHistory = [evt for evt in newTask['events'] if not evt['eventType'].startswith('Decision')]
print eventHistory

print len(eventHistory)

if eventHistory[0] != 'WorkflowExecutionStarted':
    print 'Expected freshly started workflow'
else:
    print 'Freshly started workflow'

In [ ]:
# First decision is to start the first activity
activity_id = 'activityid-' + str(uuid.uuid4())

schedule_activity(newTask['taskToken'], activities[0], activity_id)

In [ ]:
print_pending_descision_task_count()
print_pending_activity_task_count()

In [ ]:
task = client.poll_for_activity_task(
    domain=workflowDomain,
    taskList={'name': tasklist},
    identity='worker-1'
)

if 'taskToken' not in task:
    print "Poll timed out, no new task.  Repoll"
else:
    print "New task arrived"

client.respond_activity_task_completed(
    taskToken=task['taskToken'],
    result='success'
)

print "Task Done"

response = client.count_pending_activity_tasks(
    domain=workflowDomain,
    taskList={
        'name': tasklist
    }
)

print response

In [ ]:
# Is there a task available for the decider?
newTask = client.poll_for_decision_task(
    domain=workflowDomain,
    taskList={'name': tasklist},
    identity='decider-1',
    reverseOrder=False)

print newTask

In [ ]:
# Second decision is to start the second activity
activity_id = 'activityid-' + str(uuid.uuid4())

schedule_activity(newTask['taskToken'], activities[1], activity_id)

In [ ]:
poll_and_complete_task()

In [ ]:
print_pending_descision_task_count()

In [ ]:
# Is there a task available?
newTask = client.poll_for_decision_task(
    domain=workflowDomain,
    taskList={'name': tasklist},
    identity='decider-1',
    reverseOrder=False)

In [ ]:
# Schedule and complete third task
activity_id = 'activityid-' + str(uuid.uuid4())
schedule_activity(newTask['taskToken'], activities[2], activity_id)


In [ ]:
print_pending_activity_task_count()
poll_and_complete_task()
print_pending_descision_task_count()

In [ ]:
print_pending_descision_task_count()

In [ ]:
newTask = client.poll_for_decision_task(
    domain=workflowDomain,
    taskList={'name': tasklist},
    identity='decider-1',
    reverseOrder=False)

activity_id = 'activityid-' + str(uuid.uuid4())
schedule_activity(newTask['taskToken'], activities[3], activity_id)

In [ ]:
print_pending_activity_task_count()

In [ ]:
poll_and_complete_task()
print_pending_descision_task_count()

In [ ]:
newTask = client.poll_for_decision_task(
    domain=workflowDomain,
    taskList={'name': tasklist},
    identity='decider-1',
    reverseOrder=False)

print newTask

In [ ]:
response = client.respond_decision_task_completed(
        taskToken=newTask['taskToken'],
        decisions=[
          {
            'decisionType': 'CompleteWorkflowExecution',
            'completeWorkflowExecutionDecisionAttributes': {
              'result': 'success'
            }
          }
        ]
      )

print response

In [ ]:
print_pending_descision_task_count()
print_pending_activity_task_count()

In [ ]:
eventHistory = [evt for evt in newTask['events'] if not evt['eventType'].startswith('Decision')]

for event in eventHistory:
    print event
    print